In [1]:
import os
import random
import shutil

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 1. Get data

In [2]:

dataset_dir = "covid-dataset"
class_names = ['COVID', 'Normal', 'Viral Pneumonia']
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, "test")

# Make directories with each class name
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Train and test ratio
split_ratio = 0.8
covid_train_split = int(len(os.listdir("covid-dataset/COVID/images/")) * split_ratio)
normal_train_split = int(len(os.listdir("covid-dataset/Normal/images/")) * split_ratio)
pneumonia_train_split = int(len(os.listdir("covid-dataset/Viral Pneumonia/images/")) * split_ratio)

# Shuffle directories
for class_name in class_names:
    random.shuffle(os.listdir(dataset_dir + "/" + class_name + "/images"))

# Move images into train and test
covid_src_images = os.listdir("covid-dataset/COVID/images/")
train_covid_images = covid_src_images[:covid_train_split]
test_covid_images = covid_src_images[covid_train_split:]
for image in train_covid_images:
    image_path = os.path.join("covid-dataset/COVID/images/", image)
    shutil.move(image_path, "covid-dataset/train/COVID")
for image in test_covid_images:
    image_path = os.path.join("covid-dataset/COVID/images/", image)
    shutil.move(image_path, "covid-dataset/test/COVID")

normal_src_images = os.listdir("covid-dataset/Normal/images/")
train_normal_images = normal_src_images[:normal_train_split]
test_normal_images = normal_src_images[normal_train_split:]
for image in train_normal_images:
    image_path = os.path.join("covid-dataset/Normal/images/", image)
    shutil.move(image_path, "covid-dataset/train/Normal")
for image in test_normal_images:
    image_path = os.path.join("covid-dataset/Normal/images/", image)
    shutil.move(image_path, "covid-dataset/test/Normal")

pneumonia_src_images = os.listdir("covid-dataset/Viral Pneumonia/images/")
train_pneumonia_images = pneumonia_src_images[:pneumonia_train_split]
test_pneumonia_images = pneumonia_src_images[pneumonia_train_split:]
for image in train_pneumonia_images:
    image_path = os.path.join("covid-dataset/Viral Pneumonia/images/", image)
    shutil.move(image_path, "covid-dataset/train/Viral Pneumonia")
for image in test_pneumonia_images:
    image_path = os.path.join("covid-dataset/Viral Pneumonia/images/", image)
    shutil.move(image_path, "covid-dataset/test/Viral Pneumonia")

print("Files moved successfully!")



Files moved successfully!


# 2. Dataset and DataLoader

In [3]:
# HYPERPARAMETERS
BATCH_SIZE=32
NUM_WORKERS=os.cpu_count()
LEARNING_RATE=0.01
EPOCHS = 1

In [4]:
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir, transform=data_transforms)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transforms)
class_names = train_data.classes
print(class_names)
print(f"Train data: {train_data} \n Test data: {test_data}")

['COVID', 'Normal', 'Viral Pneumonia']
Train data: Dataset ImageFolder
    Number of datapoints: 12121
    Root location: covid-dataset\train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           ) 
 Test data: Dataset ImageFolder
    Number of datapoints: 3032
    Root location: covid-dataset\test
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )


In [5]:
train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
test_dataloader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)

# 2.1 Turn it into a script

In [6]:
os.makedirs("scripts", exist_ok=True)

In [7]:
%%writefile scripts/data_setup.py
"""
Putting data into Imagefolder and Dataloader
"""

import os
from torch.utils.data import DataLoader
from torchvision import datasets

def create_dataloaders(train_dir, test_dir, transform, batch_size, num_workers):
    train_data = datasets.ImageFolder(root=train_dir, transform=transform)
    test_data = datasets.ImageFolder(root=test_dir, transform=transform)

    train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    test_dataloader = DataLoader(dataset=test_data,batch_size=batch_size, num_workers=num_workers)

    class_names = train_data.classes

    return train_dataloader, test_dataloader, class_names

Overwriting scripts/data_setup.py


# 3.1 Model (CovidAid)

In [8]:

class CovidAidModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.covid_aid_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU()
        )
        self.covid_aid_2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU()
        )
        self.covid_aid_3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU()
        )
        self.covid_aid_4 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU()
        )
        self.covid_aid_5 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU()
        )
        self.covid_aid_6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU()
        )
        self.covid_aid_7 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.LeakyReLU()
        )
        
        self.covid_aid_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
        )
        self.covid_aid_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
        )
        self.covid_aid_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
        )

        self.covid_aid_block_4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
        )

        self.maxpool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.maxpool_2 = nn.MaxPool2d(kernel_size=1, stride=1)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(363, 3)

    def forward(self, x):
        x = self.covid_aid_1(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_2(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_1(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_2(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_3(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_4(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_3(x)
        x = self.covid_aid_4(x)
        x = self.covid_aid_5(x)
        x = self.covid_aid_6(x)
        x = self.covid_aid_7(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x

In [9]:
# Dummy forward pass to test if it works
# batches of images and its label
img_batch, label_batch = next(iter(train_dataloader))

# testibg model to see if it works
img_single, label_single = img_batch[0].unsqueeze(dim=0), label_batch[0]
print(f"Single image.shape: {img_single.shape}")

model_0.eval()
with torch.inference_mode():
    pred = model_0(img_single.to(device))
    
print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1)}\n")
print(f"Actual label:\n{label_single}")

Single image.shape: torch.Size([1, 3, 256, 256])


NameError: name 'model_0' is not defined

# 3.1.1 CovidAid Script mode

In [ ]:
%%writefile scripts/covid_aid.py
"""
Contains code about CovidAid Model. Original paper: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9418407
"""
import torch
from torch import nn

class CovidAidModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.covid_aid_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(8),
            nn.LeakyReLU()
        )
        self.covid_aid_2 = nn.Sequential(
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU()
        )
        self.covid_aid_3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU()
        )
        self.covid_aid_4 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU()
        )
        self.covid_aid_5 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU()
        )
        self.covid_aid_6 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU()
        )
        self.covid_aid_7 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.LeakyReLU()
        )
        
        self.covid_aid_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
        )
        self.covid_aid_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
        )
        self.covid_aid_block_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),
        )

        self.covid_aid_block_4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(),
        )

        self.maxpool_1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.maxpool_2 = nn.MaxPool2d(kernel_size=1, stride=1)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(363, 3)

    def forward(self, x):
        x = self.covid_aid_1(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_2(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_1(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_2(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_3(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_block_4(x)
        x = self.maxpool_1(x)
        x = self.covid_aid_3(x)
        x = self.covid_aid_4(x)
        x = self.covid_aid_5(x)
        x = self.covid_aid_6(x)
        x = self.covid_aid_7(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x

Overwriting scripts/covid_aid.py


# 3.2 Squeeze Net

In [16]:
import torch.nn as nn

class FireModule(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d()

    def forward(self, x):
        return x

class SqueezeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=7, stride=2, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        print(x.shape)

        return x

In [17]:
import torch
test_img = torch.rand(3, 224,224)
print(test_img.shape)
model = SqueezeNet()
model(test_img) 


torch.Size([3, 224, 224])
torch.Size([96, 55, 55])


tensor([[[ 8.0663e-01,  8.0663e-01,  7.3614e-01,  ...,  4.8381e-01,
           7.0762e-01,  6.3100e-01],
         [ 8.0663e-01,  8.0663e-01,  7.1580e-01,  ...,  6.6817e-01,
           7.2797e-01,  7.2797e-01],
         [ 6.9625e-01,  8.2464e-01,  8.2464e-01,  ...,  6.8575e-01,
           7.2797e-01,  7.2797e-01],
         ...,
         [ 6.5404e-01,  7.3019e-01,  7.7358e-01,  ...,  6.5305e-01,
           6.2561e-01,  6.2090e-01],
         [ 7.3302e-01,  7.0545e-01,  7.7358e-01,  ...,  6.5305e-01,
           6.2561e-01,  6.2090e-01],
         [ 5.3952e-01,  5.3952e-01,  3.7404e-01,  ...,  6.6434e-01,
           5.6052e-01,  4.5004e-01]],

        [[ 3.7472e-01,  2.8119e-01,  3.1155e-01,  ...,  3.2233e-01,
           3.0552e-01,  3.0201e-01],
         [ 3.0673e-01,  3.3790e-01,  2.1017e-01,  ...,  3.1381e-01,
           2.2797e-01,  3.0201e-01],
         [ 1.1941e-01,  2.5139e-01,  3.1782e-01,  ...,  3.3702e-01,
           4.9994e-01,  3.2326e-01],
         ...,
         [ 2.5118e-01,  3

# 4. Create Train and Test step

In [ ]:
import torch
import torchmetrics
accuracy_fn = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)

def train_step(model, dataloader, loss_fn, optimizer, device):
    model.train()

    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Forward
        y_pred = model(X) # returns shape [32,3]

        # Loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # optimizer
        optimizer.zero_grad()

        # backward
        loss.backward()

        # step
        optimizer.step()

        # accuracy across batch
        y_pred_class = torch.argmax(y_pred,dim=1)
        train_acc += accuracy_fn(y_pred_class, y)
        break

    # get average loss and acc per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc


In [ ]:
train_step(model_0, train_dataloader, loss_fn, optimizer, device)

In [16]:
def test_step(model, dataloader, loss_fn, device):
    model.eval()

    test_loss, test_acc = 0, 0 
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # forward
            y_pred = model(X)

            # loss
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()

            # accuracy across batch
            test_pred_label = torch.argmax(y_pred,dim=1)
            test_acc += accuracy_fn(test_pred_label, y)


    # get average loss and acc per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [17]:
def train(model, optimizer, loss_fn, epochs, device):
    # Create empty results dictionary
    results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
    }

    # Training loop
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

# 4.1 Convert to a script

In [22]:
%%writefile scripts/training.py
import torch
import torchmetrics
device = 'cuda' if torch.cuda.is_available() else 'cpu'
accuracy_fn = torchmetrics.Accuracy(task="multiclass", num_classes=3).to(device)

def train_step(model, dataloader, loss_fn, optimizer, device):
    model.train()

    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Forward
        y_pred = model(X) # returns shape [32,3]

        # Loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        
        # optimizer
        optimizer.zero_grad()

        # backward
        loss.backward()

        # step
        optimizer.step()

        # accuracy across batch
        y_pred_class = torch.argmax(y_pred,dim=1)
        train_acc += accuracy_fn(y_pred_class, y)

    # get average loss and acc per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model, dataloader, loss_fn, device):
    model.eval()

    test_loss, test_acc = 0, 0 
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # forward
            y_pred = model(X)

            # loss
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()

            # accuracy across batch
            test_pred_label = torch.argmax(y_pred,dim=1)
            test_acc += accuracy_fn(test_pred_label, y)


    # get average loss and acc per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    # Create empty results dictionary
    results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
    }

    # Training loop
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)

        print(f"Epoch {epoch} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results

Overwriting scripts/training.py


# 5. Save the model

In [36]:
import os
import torch

def save_model(model, target_dir, model_name):
    # Create directory to save models
    os.makedirs(target_dir, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth")
    model_saved_path = target_dir + '/' + model_name

    # save model
    print(f"Saved model to: {model_saved_path}")
    torch.save(model.state_dict(), model_saved_path)
    


# 5.1 Convert to script

In [3]:
%%writefile scripts/save_model.py
import os
import torch

def save_model(model, target_dir, model_name):
    # Create directory to save models
    target_dir = os.path.join("../Covid-Classificaton", target_dir)
    os.makedirs(target_dir, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pt")
    model_saved_path = target_dir + '/' + model_name

    # save model
    print(f"Saved model to: {model_saved_path}")
    torch.save(model.state_dict(), model_saved_path)

Overwriting scripts/save_model.py


# 6. Train, Evaluate & Save model

In [33]:
from torchmetrics.classification import MulticlassAccuracy
from timeit import default_timer as timer

# HYPERPARAMETERS
SEED=42
BATCH_SIZE=32
NUM_WORKERS= 4 #os.cpu_count()
LEARNING_RATE=0.01
EPOCHS = 5

# Instantiniate seeds
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# MODELS
model_0 = CovidAidModel().to(device)

# Loss Function and Optimizer
loss_fn = nn.CrossEntropyLoss()
accuracy_fn = MulticlassAccuracy(num_classes=3)
optimizer = torch.optim.SGD(model_0.parameters(), lr=LEARNING_RATE)

# Start timer
start_time = timer()

# Train models
model_0_results = train(model_0, optimizer, loss_fn, EPOCHS, device)

# End timer
end_time = timer()
print(f"Total training time: {end_time-start_time:.2f} seconds")

# Save Model
save_model(model_0, target_dir='models', model_name='CovidAid.pth')

NameError: name 'train' is not defined

# 6.1 Convert to Script

In [2]:
%%writefile scripts/executable.py
from timeit import default_timer as timer
import os
import torch.nn as nn
import torch
import torchvision.transforms as transforms
import data_setup, covid_aid, save_model, training

# Varaibles
train_dir = "../covid-dataset/train/"
test_dir = "../covid-dataset/test/"
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor()
])

# HYPERPARAMETERS
SEED=42
BATCH_SIZE=32
NUM_WORKERS=4 #os.cpu_count()
LEARNING_RATE=0.01
EPOCHS = 5
print(f"Learning Rate: {LEARNING_RATE} | Number of Epochs: {EPOCHS}")

# Instantiniate seeds
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# DATA
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir, test_dir, data_transforms, BATCH_SIZE, NUM_WORKERS)

# MODELS
model_0 = covid_aid.CovidAidModel().to(device)

# Loss Function and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr=LEARNING_RATE)

# Start timer
start_time = timer()

# Train models
model_0_results = training.train(model_0, train_dataloader, test_dataloader, optimizer, loss_fn, EPOCHS, device)

# End timer
end_time = timer()
print(f"Total training time: {end_time-start_time:.2f} seconds")

# Save Model
save_model.save_model(model_0, target_dir='models', model_name='CovidAid.pt')

Overwriting scripts/executable.py
